# 최종 테스트용 코드 (셀 1개에 단계별로 코드 묶어놓음)

- **고려사항**
  - **연령대**
    - 데이터 특성 상 20대,30대의 검색 인기도가 대체로 높은 편이기 때문에 20,30대 데이터가 우세한 경향이 있음
    - 그래도 서비스 이용자를 생각한다면 20,30대 연령대의 추천 결과가 잘 나오기 때문에 괜찮을 것 같다.
    - 다른 연령대는 잘 안될것같음
    - **성별이랑 똑같은 방식으로 100 나눠서 스케일링 중인데 혹시 다른 방법 없을까**
  - **성별**
    - 남,녀 고르게 잘 나오는편 같음
    - 남자가 압도적으로 높은 장소가 거의 안보임 (내가 못본 걸수도)
    - 남성 고르면 보통 남녀비율 비슷비슷한데, 여성 고르면 여자 비율 높은 장소들이 잘 추천됨
  - **MBTI**
    - IN이 압도적으로 많음
    - MBTI 유형 상 리뷰를 IN유형의 사람들이 많이 작성하기 때문에 IN으로 분류가 많이 되는 것 같다.
    - 리뷰 데이터를 기반으로 분류한 결과이다 보니 IN쪽에 쏠리는 상황은 어쩔 수 없을지도?
  - **키워드**
    - 그래도 고른대로 잘 나오는 편인 것 같다.
    - 근데 음식점 같은 곳이 다른 키워드에 비해 훨씬 많기 때문에 Top N개보단 그냥 유사도 순서대로 전부 정렬하고 사용자가 내부에서 다시 필터링한대로 볼 수 있도록 해줘야 할 것 같음.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
pd.set_option('display.max_columns', None) # 전체 열 보기

### **MBTI 데이터 불러오기**

In [3]:
mbti = pd.read_csv('final_mbti.csv')
temp = mbti.drop(columns=['Unnamed: 0'])

# 장소마다 리뷰 데이터의 MBTI 분포 개수 세기
temp2 = list(temp.columns) # 칼럼 하나씩 ['review1', 'review2', 'review3', ...]
IS__, IN__, ES__, EN__ = [], [], [], []

for idx, row in temp.iterrows():
    IS__.append(0)
    IN__.append(0)
    ES__.append(0)
    EN__.append(0)
    for t in temp2:
        if row[t] == 'IS':
            IS__[idx] += 1
        elif row[t] == 'IN':
            IN__[idx] += 1
        elif row[t] == 'ES':
            ES__[idx] += 1
        elif row[t] == 'EN':
            EN__[idx] += 1
        else:
            pass

mbti_count = pd.DataFrame({
    'IS':IS__,
    'IN':IN__,
    'ES':ES__,
    'EN':EN__,
})

temp = pd.concat([temp, mbti_count], axis=1)
print("========================[장소 별 리뷰 MBTI 개수 카운트 결과]========================")
display(temp)

# 각 장소별 리뷰 총합
mbti_count['mbti_sum'] = mbti_count.T.sum()

## 각 장소마다 MBTI 유형을 총합 1이 되는 비율로 환산
mbti_count['mbti_is'] = mbti_count['IS'] / mbti_count['mbti_sum']
mbti_count['mbti_in'] = mbti_count['IN'] / mbti_count['mbti_sum']
mbti_count['mbti_es'] = mbti_count['ES'] / mbti_count['mbti_sum']
mbti_count['mbti_en'] = mbti_count['EN'] / mbti_count['mbti_sum']
print("========================[장소 별 리뷰 MBTI 비율 환산]========================")
display(mbti_count)

# MBTI 비율만 남기기
mbti_final = mbti_count[['mbti_is', 'mbti_in', 'mbti_es', 'mbti_en']]

print("========================[결측치 확인]========================")
print(mbti_final.isna().sum())
# NaN값 0으로 채워주기
mbti_final = mbti_final.fillna(0)
print("========================[최종 MBTI 비율 데이터]========================")
display(mbti_final)

========================[장소 별 리뷰 MBTI 개수 카운트 결과]========================


,review1,review2,review3,review4,review5,review6,review7,review8,review9,review10,review_blog1,review_blog2,review_blog3,review_blog4,review_blog5,review_blog6,review_blog7,review_blog8,review_blog9,review_blog10,IS,IN,ES,EN
0,EN,IN,ES,EN,IN,IN,IN,IN,IN,IN,IN,IS,IN,IN,IN,IN,IN,IN,EN,ES,1,14,2,3
1,EN,IN,IS,IN,ES,IN,IN,EN,EN,EN,IN,ES,IN,IN,IN,ES,IN,IN,IN,IN,1,12,3,4
2,IN,EN,IN,IN,IN,IN,IN,IN,IN,IN,IN,IN,IN,IN,ES,IN,IN,IN,EN,IN,0,17,1,2
3,IN,IN,IN,EN,ES,IN,IN,IN,IN,IN,IN,IN,ES,IN,IN,IN,IN,EN,IN,ES,0,15,3,2
4,IN,EN,IN,IN,IN,IN,IN,IN,EN,EN,EN,EN,IN,ES,IN,IN,IN,IN,IN,IN,0,14,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12719,IS,IN,IS,IS,IS,IS,no data,no data,no data,no data,ES,IN,IN,IN,no data,no data,no data,no data,no data,no data,5,4,1,0
12720,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,0,0,0,0
12721,ES,no data,no data,no data,no data,no data,no data,no data,no data,no data,IN,IN,IN,no data,no data,no data,no data,no data,no data,no data,0,3,1,0
12722,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,IS,IN,IN,ES,IN,IN,IN,EN,EN,no data,1,5,1,2


========================[장소 별 리뷰 MBTI 비율 환산]========================


,IS,IN,ES,EN,mbti_sum,mbti_is,mbti_in,mbti_es,mbti_en
0,1,14,2,3,20,0.050000,0.700000,0.100000,0.150000
1,1,12,3,4,20,0.050000,0.600000,0.150000,0.200000
2,0,17,1,2,20,0.000000,0.850000,0.050000,0.100000
3,0,15,3,2,20,0.000000,0.750000,0.150000,0.100000
4,0,14,1,5,20,0.000000,0.700000,0.050000,0.250000
...,...,...,...,...,...,...,...,...,...
12719,5,4,1,0,10,0.500000,0.400000,0.100000,0.000000
12720,0,0,0,0,0,NaN,NaN,NaN,NaN
12721,0,3,1,0,4,0.000000,0.750000,0.250000,0.000000
12722,1,5,1,2,9,0.111111,0.555556,0.111111,0.222222


========================[결측치 확인]========================
mbti_is    745
mbti_in    745
mbti_es    745
mbti_en    745
dtype: int64
========================[최종 MBTI 비율 데이터]========================


,mbti_is,mbti_in,mbti_es,mbti_en
0,0.050000,0.700000,0.100000,0.150000
1,0.050000,0.600000,0.150000,0.200000
2,0.000000,0.850000,0.050000,0.100000
3,0.000000,0.750000,0.150000,0.100000
4,0.000000,0.700000,0.050000,0.250000
...,...,...,...,...
12719,0.500000,0.400000,0.100000,0.000000
12720,0.000000,0.000000,0.000000,0.000000
12721,0.000000,0.750000,0.250000,0.000000
12722,0.111111,0.555556,0.111111,0.222222


## **장소 데이터 불러오기**
- 연령대 스케일링 방식 다르게 해볼만한 방법 있을지?

In [4]:
# 검색할 지역구 지정
REGION = '마포구'

In [5]:
# 장소 데이터에 MBTI 데이터 합치기
place = pd.read_csv('total_place.csv')
place = pd.concat([place, mbti_final], axis=1)

# 지역구 선택
place = place.loc[place['gu']==REGION]
print(f"========================[{REGION} 지역 장소들입니다.]========================")
print(place['search_key'].value_counts())

# 추천에 사용할 정보만 빼기
df = place[['place_num', 'search_key', 'year_10', 'year_20', 'year_30', 'year_40', 'year_50', 'year_60',
            'male', 'female', 'mbti_is', 'mbti_in', 'mbti_es', 'mbti_en']]

# 키워드 => 가변수화
df2 = pd.get_dummies(df, columns=['search_key'])

# 장소id 분리
place_id = df2['place_num']
place_value = df2.drop(columns=['place_num'])

# 가변수화 후 칼럼명 유저 입력정보와 맞춰주기
place_value.columns = [
    'year_10','year_20','year_30','year_40','year_50','year_60',
    'gender_male','gender_female', 'mbti_is', 'mbti_in', 'mbti_es', 'mbti_en',
    'walking_street','leisure_craft','walking_park','leisure_theater','cafe_dessert',
    'cafe_bakery','leisure_department','restaurant_bunsik','leisure_book','walking_market',
    'restaurant_west','restaurant_japan','leisure_gallery','restaurant_china','cafe_cafe',
    'leisure_popup','restaurant_fast','restaurant_korea',]

# [[[스케일링]]]
a = place_value[['year_10', 'year_20', 'year_30', 'year_40', 'year_50', 'year_60', 'gender_male', 'gender_female']]
b = place_value.drop(columns=['year_10', 'year_20', 'year_30', 'year_40', 'year_50', 'year_60', 'gender_male', 'gender_female'])

## 최소(0)~최대(100) 이 되도록 스케일링
# 연령대 : 각 칼럼별로 0~100을 0~1로
# 성별 : 남녀 합이 1이 되도록 0~1로
a = a / 100

place_value_mm = pd.concat([a, b], axis=1)

# 장소id 다시 합쳐주기
df3 = pd.concat([place_id, place_value_mm], axis=1)

# 데이터프레임 칼럼 순서 다시 조정해주기
df3 = df3[['place_num','year_10','year_20','year_30','year_40','year_50','year_60',
    'gender_male','gender_female', 'mbti_is', 'mbti_in', 'mbti_es', 'mbti_en',
    'restaurant_korea','restaurant_west','restaurant_china','restaurant_japan','restaurant_fast','restaurant_bunsik',
    'cafe_cafe','cafe_dessert','cafe_bakery',
    'leisure_gallery','leisure_craft','leisure_popup','leisure_theater','leisure_book','leisure_department',
    'walking_park','walking_market','walking_street',
]]

print(f"========================[스케일링, 칼럼 순서 맞추기 작업 완료]========================")
display(df3)

========================[마포구 지역 장소들입니다.]========================
한식       203
일식       202
중식       202
분식       199
공방       198
양식       197
베이커리     187
극장       183
서점       181
디저트카페    150
패스트푸드     88
전시관       86
공원        64
시장        53
카페        46
거리        39
복합쇼핑몰      7
팝업스토어      5
Name: search_key, dtype: int64
========================[스케일링, 칼럼 순서 맞추기 작업 완료]========================


,place_num,year_10,year_20,year_30,year_40,year_50,year_60,gender_male,gender_female,mbti_is,mbti_in,mbti_es,mbti_en,restaurant_korea,restaurant_west,restaurant_china,restaurant_japan,restaurant_fast,restaurant_bunsik,cafe_cafe,cafe_dessert,cafe_bakery,leisure_gallery,leisure_craft,leisure_popup,leisure_theater,leisure_book,leisure_department,walking_park,walking_market,walking_street
7539,1811663461,0.2500,0.8557,0.8580,0.8478,0.7408,0.6638,0.53,0.47,0.20,0.400000,0.200000,0.200000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7540,1216931278,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.842105,0.105263,0.052632,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7541,18799565,0.2231,0.8543,0.7478,0.5304,0.5590,0.4819,0.52,0.48,0.05,0.850000,0.050000,0.050000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7542,35523842,0.1740,0.8221,0.6911,0.5073,0.5401,0.4575,0.54,0.46,0.05,0.800000,0.050000,0.100000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7543,31558719,0.2085,0.3442,0.2236,0.2968,0.3413,0.2290,0.54,0.46,0.10,0.500000,0.200000,0.200000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9824,18744851,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.20,0.600000,0.100000,0.100000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9825,31148576,0.1754,0.3465,0.2020,0.1830,0.1794,0.1821,0.47,0.53,0.00,0.700000,0.100000,0.200000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9826,18822133,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9827,33997144,0.0520,0.1556,0.1514,0.2210,0.3268,0.4142,0.50,0.50,0.00,0.700000,0.200000,0.100000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## **테스트용 사용자 데이터 만들기**

In [3]:
# 연령대 : 20대
# 성별 : 남성
# MBTI : IS
# 키워드 : 한식, 일식, 카페, 갤러리, 극장, 공원
userform = pd.DataFrame({
    'age_10': [0.0],
    'age_20': [1.0],
    'age_30': [0.0],
    'age_40': [0.0],
    'age_50': [0.0],
    'age_60': [0.0],
    'gender_male': [1.0],
    'gender_female': [0.0],
    'mbti_is':[1.0],
    'mbti_in':[0.0],
    'mbti_es':[0.0],
    'mbti_en':[0.0],
    'restaurant_korea': [1.0],
    'restaurant_west': [0.0],
    'restaurant_china': [0.0],
    'restaurant_japan': [1.0],
    'restaurant_fast': [0.0],
    'restaurant_bunsik': [0.0],
    'cafe_cafe': [1.0],
    'cafe_dessert': [0.0],
    'cafe_bakery': [0.0],
    'leisure_gallery': [1.0],
    'leisure_craft': [0.0],
    'leisure_popup': [0.0],
    'leisure_theater': [1.0],
    'leisure_book': [0.0],
    'leisure_department': [0.0],
    'walking_park': [1.0],
    'walking_market': [0.0],
    'walking_street': [0.0],
})
userform

,age_10,age_20,age_30,age_40,age_50,age_60,gender_male,gender_female,mbti_is,mbti_in,mbti_es,mbti_en,restaurant_korea,restaurant_west,restaurant_china,restaurant_japan,restaurant_fast,restaurant_bunsik,cafe_cafe,cafe_dessert,cafe_bakery,leisure_gallery,leisure_craft,leisure_popup,leisure_theater,leisure_book,leisure_department,walking_park,walking_market,walking_street
0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


## **유사도 측정**

In [4]:
# # 유사도 상위 몇 개의 장소를 뽑을지 지정
# TOP_N = 300

recParams_df = pd.read_csv('DB용데이터/찐막/final_DBrecommend_params.csv')
places_df = pd.read_csv('DB용데이터/찐막/final_DBplaces_clean.csv')

In [5]:
# 코사인 유사도 구하기 [사용자 입력 정보 - 장소 데이터 간의]
recommend_places = pd.DataFrame(cosine_similarity(
    userform,
    recParams_df.loc[:, recParams_df.columns != 'place_code']),
    columns = list(recParams_df.place_code), index=['similarity']
)

# 유사도 상위 N개만큼의 장소만큼
topN = recommend_places.T.sort_values('similarity', ascending=False)#[:TOP_N]
# Top N 장소 id 추출
topN_df = pd.DataFrame({
    'place_code': list(topN.index)
})
# 장소id로 장소 데이터와 INNER JOIN
final_recommends = pd.merge(topN_df, places_df, how='left', on='place_code')
print(f"========================[유사도 순으로 추천된 장소들입니다.]========================")
display(final_recommends)

print(f"========================[방문자 수, 평점 높은 순서대로 보기]========================")
display(final_recommends.sort_values(by=['review_visitor_count', 'rating'], ascending=False))

print(f"========================[장소의 종류 분포 확인]========================")
print(final_recommends['search_category'].value_counts())

========================[유사도 순으로 추천된 장소들입니다.]========================


,place_code,name,photo,search_region,search_category,category,address,nearest_hotplace,rating,tel,age_10,age_20,age_30,age_40,age_50,age_60,gender_male,gender_female,mbti_is,mbti_in,mbti_es,mbti_en,review_summary1,review_summary1_cnt,review_summary2,review_summary2_cnt,review_summary3,review_summary3_cnt,review1,review2,review3,review4,review5,review6,review7,review8,review9,review10,review_blog_count,review_visitor_count,review_keywords
0,18361327,이자카야무세이,https://search.pstatic.net/common/?autoRotate=...,마포구,일식,이자카야,서울 마포구 어울마당로 127-1,홍대 관광특구,4.36,02-333-8174,28.52,49.42,35.82,28.76,20.32,11.76,53,47,0.40,0.350000,0.200000,0.050000,음식이 맛있어요,77.0,친절해요,39.0,대화하기 좋아요,24.0,음식도 맛있고 분위기도 좋고 매우 좋습니다,직원 교육부족으로 굉장히 불쾌했음. 좋게 설명 해주면 되는데 태도가 너무 했어요 \...,좁은 매장이 층층이 있더라구요. 계단으로 4층까지 올라갔.. ㅋ 타다키랑 아사히 생...,요리는 맛있었습니다만 직원이 조금 불친절해서 기분이 좋지 않았어요 일하기 싫어보였습니다,안주가 맛있어요 ㅎㅎ 게도 실하고 국물도 맛있어서 싹싹 긁어먹었어요~!!,음식이 맛있어요 그런데 주말에는 2시간 제한있네요 ㅠㅠ,술안주들이 기가막히네요~~,2022년 9월22일 저녁에 방문\n버터명란계란말이 1개와 참이슬 후레쉬 1병주문\...,친구와 함께 한잔 하려고 들어온 이자카야\n별생각없이 시켰다가 깜놀~~ 가성비 갑입...,치즈 계란말이 강추요ㅠㅠ,203,260,['좋다' '맛있다']
1,20937137,말뚝곱창1호점,https://search.pstatic.net/common/?autoRotate=...,구로구,한식,"곱창,막창,양",서울 구로구 시흥대로 571,구로디지털단지역,0.00,02-855-5298,28.80,47.41,42.11,34.38,20.87,17.96,51,49,0.45,0.350000,0.100000,0.100000,음식이 맛있어요,96.0,친절해요,29.0,재료가 신선해요,27.0,순두부찌개가 기본반찬으로 나오는데 진짜 존맛 ㅠㅠ음식은 초벌 되어 나오니 조금만 기...,볶음밥이 너무 짜서 아쉬워요,남친이 좋아해요 근데 남친이 늦게 끝나는 편이라 너무 늦으면 못가서 아쉬워요 ㅜㅜ,역 바로 앞에 있는 본점? 인데 역시 맛있습니다! 화장실이 조금 불편하지만ㅜ 곱창은...,오랜만에 구디에서 친구들과 만나서 갔는데\n\n여전히 맛있네요\n\n번창하세요,"볶음밥이 좀 많이 짜요,,",정말 맛있어요,볶음밥이 좀 짜서 아쉽고 곱창은 쏘쏘,볶음밥이 좀 아쉬웠어요ㅜ,친절하고 맛있어요!!!!,384,349,['짜다' '아쉽다' '볶음밥' '맛있다' '곱창']
2,457840624,홍곱창 숙명여대점,https://search.pstatic.net/common/?autoRotate=...,용산구,한식,"곱창,막창,양",서울 용산구 청파로43가길 31 1층,서울역,4.56,02-707-0092,30.16,49.09,33.88,26.44,25.21,11.76,46,54,0.40,0.350000,0.050000,0.200000,음식이 맛있어요,185.0,친절해요,61.0,특별한 메뉴가 있어요,39.0,소금구이 막창 꼭 드세요 바삭하고 속은 부드럽고 소금톡톡 찍먹하면 엄청 고소해요!!...,대학생때 회기점에서 자주 먹었었는데 숙대 근처에 왔다가 반가워서 방문했어요!\n\n...,https://m.blog.naver.com/kdw6893 / 블로그 참조 웨이팅이...,데리야끼 막창 너무 맛있어요! 다음에 또 방문하고싶습니다.,곱창 잘 못 먹는데 홍곱창 곱창은 정말 잘 먹어요. \n쫄깃쫄깃 상태 좋아서 비린 ...,빨리 나가라고 눈치를 주긴 하지만.. 맛있어요,데리야끼막창맛집,불곱창 맛있어요.단골이에요 .친구들이랑 한잔하면서 막창 곱창 먹으면서 즐기기 좋아요,데리야끼 꼭 드세요!!,네이버 맛집 추천으로 방문했는데 특히 데리야끼막창\n맛있어요^^,238,590,['좋다' '소금' '맛있다' '막창' '데리야끼']
3,1449590332,할리스 구로역점,https://search.pstatic.net/common/?autoRotate=...,구로구,카페,카페,서울 구로구 새말로 18-29,신도림역,4.44,02-862-3216,25.00,95.77,76.59,25.00,25.00,25.00,39,61,0.50,0.333333,0.111111,0.055556,커피가 맛있어요,57.0,친절해요,38.0,매장이 청결해요,34.0,흑임자 크림 라떼 시켰고요 브로마이드 사진에는 흑임자 크림이 듬뿍이고 3-4cm는 ...,최곱니당~~,"할리스는 바닐라딜라이가 가장 맛있어요,,, 신메뉴 드시고 싶으신 분들은 흑임자 드세...",좋아요,굿,굿,좋아요,복숭아음료 맛있어요 :-),NaN,NaN,19,626,['흑임자' '크림' '맛있다']
4,1581871973,경성주막1929 경복궁역점,https://search.pstatic.net/common/?autoRotate=...,종로구,일식,이자카야,서울 종로구 자하문로 3 2층,경복궁·서촌마을,4.14,02-737-2445,11.76,47.46,28.51,26.33,11.76,11.76,47,53,0.30,0.500000,0.100000,0.100000,음식이 맛있어요,47.0,가성비가 좋아요,29.0,친절해요,26.0,경복궁역에서 가볍게 한잔하기 좋은 곳!\n저렴해서 이것저것 시켜봤는데 (오코노미야끼...,안주가 맛있어요,간단히 홍합오뎅탕 ~~\n여러가지 골라먹는 어묵맛이 좋아요\n전철역도 가깝고 창밖을...,음 애매한데 그래도 저렴하고 맛있는편,시원한 맥주한잔하려다 \n마음에드는 안주 발견,파닭꼬치는 말라있고 명란은 짜요 ㅜㅜ 파인애플이 젤 맛났네요,안주가 맛이 없기가 쉽지않은데\n여긴 안주가 너무 맛이 없음 ..\n음식 다 잘먹고...,결론부터 말하면 진짜 인생 최악 술집 중 한곳 입니다.\n들어갔을때 아들로 보이는 ...,안주다양하고 맛있어요 :),매장이 넓고 좌석이 많아서 좋네요!!\n사이드도 다양해서 좋습니닷 굿굿,20,222,['최악' '좋다' '안주' '명란' '맛있다']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12087,11540107,북클럽 이달의책,https://search.pstatic.net/common/?autoRotate=...,마포구,서점,서점,서울 마포구 양화로23길 22-7,홍대 관광특구,0.00,02-326-3638,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.000000,0.000000,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,no result
12088,1822083228,평화도서관 나무,https://search.pstatic.net/common/?autoRotate=...,마포구,서점,서점,서울 마포구 월드컵로25길 55,망원한강공원,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.000000,0.000000,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0

========================[방문자 수, 평점 높은 순서대로 보기]========================


,place_code,name,photo,search_region,search_category,category,address,nearest_hotplace,rating,tel,age_10,age_20,age_30,age_40,age_50,age_60,gender_male,gender_female,mbti_is,mbti_in,mbti_es,mbti_en,review_summary1,review_summary1_cnt,review_summary2,review_summary2_cnt,review_summary3,review_summary3_cnt,review1,review2,review3,review4,review5,review6,review7,review8,review9,review10,review_blog_count,review_visitor_count,review_keywords
8149,12183349,교보문고 광화문점,https://search.pstatic.net/common/?autoRotate=...,종로구,서점,서점,서울 종로구 종로 1 교보생명빌딩 지하1층,광화문·덕수궁,4.50,1544-1900,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.421053,0.368421,0.105263,0.105263,책 종류가 다양해요,10053.0,베스트셀러가 많아요,5946.0,신간이 빨리 들어와요,5793.0,"요즘같이 추운 날, \n지하철 통로와 연결되어 있다는 점이 큰 메리트에요\n\n책 ...","주일 오전부터 책 사는, 읽는 사람들로 북적이는 교보",굿,사람이 너무너무너무너무 많아서 개인적으로는 잠실 교보가 좋긴한데 여기는 분위기는 더...,자주갑니다. 좋아요,좋아요,구경하는 재미가 있어요ㅎㅎ,굿,친절해요,NaN,3848,41754,['좋다']
8418,31982286,현대시티아울렛 가산점,https://search.pstatic.net/common/?autoRotate=...,구로구,복합쇼핑몰,아울렛,서울 금천구 디지털로10길 9,구로디지털단지역,4.34,02-2136-2233,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.350000,0.350000,0.200000,0.100000,친절해요,62.0,종류가 다양해요,44.0,주차하기 편해요,41.0,친절해요,완전 맛있어용 친절하시구~~ 토핑도 다양해서 짱 조음,프레첼 사먹었어용,조아요,이쁜 옷 저렴하게 샀습니다. 감사합니다!!,망했어요.. 새로 산 옷이 줄어듦..ㅠㅠ,굿,가성비 짱,친절하고 맛좋아요~,주차장 정보가 부정확해서 정확한 정보를 알려드립니다. 최초 30분 무료주차 + 이후...,1230,17440,['친절하다' '최초' '주차' '정확하다' '정보']
3708,895457986,노티드 청담,https://search.pstatic.net/common/?autoRotate=...,강남구,카페,카페,서울 강남구 도산대로53길 15 1층,압구정로데오거리,4.44,0507-1426-9399,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.050000,0.700000,0.150000,0.100000,디저트가 맛있어요,5519.0,특별한 메뉴가 있어요,1867.0,친절해요,1439.0,오랜만에 방문했는데 많이 바꼈더라구요 매장도 엄청 깔끔해지고 예뻐졌습니다ㅎㅎ 그리고...,"원래 서래점 자주 가는데 이날 근처 갈일 있어 청담점으로 방문 했어요, 서래는 도넛...",노티드는 다른 매장이랑 비교해봐도 크림을 진짜깔끔하게 잘하는것 같아요. 술마신 다음...,너무 이쁜 노티드 청담점♡♡♡♡♡\n들어가는 순간부터 동화속으로 들어가는 기분 내돈...,9개월 만에 다시 온 노티드 청담점🥰\n주말 오후라 대기 줄이 좀 있길래 오래 걸릴...,우유생크림도넛이 가장 맛있어요\n노티드는 우유크림 맛집\n할로윈 음료 팥팥팥 콜드브...,주문하고 한참 기다렸었요\n기다리다가니 시나몬 트리 라떼가 안나오고..\n또 기다리...,언제와도 기분좋은 노티드...💓\n매장 곳곳 안 예쁜 장소가 없고요!!맛은 말모.....,주말에 가서 줄이 많을까봐 걱정했는데 줄도 별로 없고 너무 예쁘게 꾸며져있어서 좋았...,평일 4시쯤 갔더니 웨이팅도 없고 자리도 꽤 널널해서 좋았어요! 도넛은 테이크아웃으...,6087,17170,['크림' '케이크' '좋다' '음료' '우유']
7053,11714659,태극당,https://search.pstatic.net/common/?autoRotate=...,종로구,베이커리,베이커리,서울 중구 동호로24길 7,종로·청계 관광특구,4.37,02-2279-3152,9.21,32.67,34.65,33.81,28.12,20.24,42,58,0.150000,0.700000,0.000000,0.150000,빵이 맛있어요,5853.0,특별한 메뉴가 있어요,2071.0,매장이 청결해요,1136.0,정말 옛날 느낌 그대로ㅎㅎ\n좀 바뀌기야 했지만\n어쩜 케이크도 이리 촌스럽게ㅎㅎ\...,일요일 4시 이후에 가니 꽤 많은 종류의 빵이 판매완료되었네요 ㅎ ㅎ\n\n야채사라...,추운날 저녁 태극당 방문했어요~\n동대입구역 2번출구에서 바로 보이네요~\n스테디셀...,배구보러 장충체육관 가면서 들렀는데\n오래된 백년빵집 빵들이 너무 맛있었어요~\n시...,동대입구. 서울에서 가장 오래된 빵집!☞태극당!\n1\n흑임자월병(3.0). 유자월...,아이스크림 모나카 먹고싶은 소망 이뤘네요 ㅎ\n너무 기대했는지 시모나랑 차이 못느끼...,"옛날향수 물씬 풍기는 버터케이크, 사라다빵,카스테라, 찹쌀떡등도 있지만 커피츄러스,...",어릴 때 생각이 나는 빵들이 있어 좋아요. 동대입구역에 바로 있어서 위치도 좋아요.,티비에서 보고 꼭 가보고 싶었던 곳인데 이번에 가보게 됨\n모나카 아이스크림 기대만...,오랜만에 들렀는데 여전합니다 빵은 계속 팔리고 새로운 빵이 계속 나타납니다 식사를 ...,3234,15892,['흑임자' '태극' '좋다' '유자' '월병']
8263,1791636481,아이파크몰,https://search.pstatic.net/common/?autoRotate=...,마포구,복합쇼핑몰,"쇼핑센터,할인매장",서울 용산구 한강대로23길 55,신촌·이대역,4.43,02-2012-0101,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.388889,0.277778,0.111111,0.222222,친절해요,756.0,종류가 다양해요,686.0,매장이 넓어요,654.0,고소하고 맛있어요.,좋아요,굿,좋아요,매번 오는곳인데 좋아요,좋아요,굿,마약옥수수빵집이에요,NaN,NaN,3351,15705,['좋다']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12080,18304916,태솔,https://search.pstatic.net/common/?autoRotate=...,마포구,서점,서점,서울 마포구 삼개로 20,신촌·이대역,0.00,02-501-4482,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.000000,0.000000,0.000000,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,no result
12082,1250021608,북앤리브,https://search.pstatic.net/common/?autoRotate=...,마포구,서점,서점,서울 마포구 망원로7길 31-18,망원한강공원,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.000000,0.000000,0.000000,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,no result
12083,1341641620,오무냐미,https://search.pstatic.net/commo

========================[장소의 종류 분포 확인]========================
한식       1199
분식       1182
일식       1159
중식       1127
양식       1124
베이커리     1058
공방        955
서점        843
디저트카페     761
극장        760
공원        411
패스트푸드     350
전시관       311
시장        296
카페        291
거리        181
팝업스토어      53
복합쇼핑몰      31
Name: search_category, dtype: int64


In [9]:
final_recommends.loc[(final_recommends['search_region'] == "강남구") & ( (final_recommends['search_category'] == "공원") | 
                                                                    (final_recommends['search_category'] == "시장") |
                                                                    (final_recommends['search_category'] == "거리") )]

,place_code,name,photo,search_region,search_category,category,address,nearest_hotplace,rating,tel,age_10,age_20,age_30,age_40,age_50,age_60,gender_male,gender_female,mbti_is,mbti_in,mbti_es,mbti_en,review_summary1,review_summary1_cnt,review_summary2,review_summary2_cnt,review_summary3,review_summary3_cnt,review1,review2,review3,review4,review5,review6,review7,review8,review9,review10,review_blog_count,review_visitor_count,review_keywords
69,1833367621,세곡해오름근린공원,https://search.pstatic.net/common/?autoRotate=...,강남구,공원,근린공원,서울 강남구 헌릉로590길 55,선릉역,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1.000000,0.000000,0.000000,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,좋아요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,no result
70,33832252,우리동네숲4호,https://search.pstatic.net/common/?autoRotate=...,강남구,공원,근린공원,서울 강남구 대치동 979-1,강남 MICE 관광특구,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1.000000,0.000000,0.000000,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,굿,굿,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,no result
75,1711182136,래미안대치팰리스공원,https://search.pstatic.net/common/?autoRotate=...,강남구,공원,근린공원,서울 강남구 대치동 1027-1,선릉역,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1.000000,0.000000,0.000000,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,굿,굿,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,no result
81,1755017308,세곡공원,https://search.pstatic.net/common/?autoRotate=...,강남구,공원,근린공원,서울 강남구 세곡동 산50-10,강남 MICE 관광특구,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1.000000,0.000000,0.000000,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,굿,좋아요,굿,굿,굿,굿,굿,좋아요,좋아요,NaN,1,10,['좋다']
109,18847710,압구정은행공원,https://search.pstatic.net/common/?autoRotate=...,강남구,공원,근린공원,서울 강남구 압구정동 459,압구정로데오거리,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.818182,0.181818,0.000000,0.000000,뷰가 좋아요,23.0,산책로가 잘 되어있어요,9.0,가격이 합리적이에요,5.0,서브웨이는 항상 맛집입니다,좋아요!!,ㅇㅇㅇ,굿,좋아요,I,굿,굿,굿,굿,1,45,no result
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11792,18833695,건대로데오거리,https://search.pstatic.net/common/?autoRotate=...,강남구,거리,도보코스,서울 광진구 자양동 8-5,압구정로데오거리,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000,0.900000,0.000000,0.100000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,0,no result
11830,1378322420,양재천카페거리,https://search.pstatic.net/common/?autoRotate=...,강남구,거리,도보코스,서울 서초구 양재동 99-3,강남역,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000,0.800000,0.100000,0.100000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133,0,no result
11992,37282496,자양동중국음식골목,https://search.pstatic.net/common/?autoRotate=...,강남구,거리,먹자거리,서울 광진구 자양동,강남 MICE 관광특구,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000,0.714286,0.142857,0.142857,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0,no result
11993,37770209,성내동주꾸미골목,https://search.pstatic.net/common/?autoRotate=...,강남구,거리,먹자거리,서울 강동구 성내동,강남 MICE 관광특구,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000,0.700000,0.300000,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63,0,no result


In [18]:
final_recommends.loc[(final_recommends['search_category']=="한식") |
                     (final_recommends['search_category']=="양식") |
                     (final_recommends['search_category']=="중식") |
                     (final_recommends['search_category']=="일식") |
                     (final_recommends['search_category']=="패스트푸드") |
                     (final_recommends['search_category']=="분식")]['search_category'].value_counts()

한식       1199
분식       1182
일식       1159
중식       1127
양식       1124
패스트푸드     350
Name: search_category, dtype: int64

In [22]:
final_recommends.sort_values(by='rating', ascending=False)

,place_code,name,photo,search_region,search_category,category,address,nearest_hotplace,rating,tel,age_10,age_20,age_30,age_40,age_50,age_60,gender_male,gender_female,mbti_is,mbti_in,mbti_es,mbti_en,review_summary1,review_summary1_cnt,review_summary2,review_summary2_cnt,review_summary3,review_summary3_cnt,review1,review2,review3,review4,review5,review6,review7,review8,review9,review10,review_blog_count,review_visitor_count,review_keywords
10211,1273737545,마쁘띠메종,https://search.pstatic.net/common/?autoRotate=...,용산구,양식,프랑스음식,서울 용산구 원효로 141-1 2층,용산역,5.0,0507-1321-4664,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.100000,0.450000,0.250000,0.200000,음식이 맛있어요,39.0,특별한 날 가기 좋아요,34.0,특별한 메뉴가 있어요,27.0,프로포즈하려고 했는데 사장님께서 너무 배려를 잘 해주셔서 성공하고 왔습니다ㅎㅎ \n...,프라이빗 테이블이라 좋았어요! 맛있었구요 😋\n기념일에 남자친구와 방문했는데 둘 다...,용산 근처에 6명이 모일 수 있는 공간을 찾다가 마쁘띠메종을 알게 되어서 예약했어요...,아늑한 분위기에 원테이블 레스토랑이에요~ 코스메뉴도 너무 알차고 시즌음료 완젼 달달...,모든 게 완벽했던 용산의 보석같은 프렌치 맛집에서의 한 끼! 잘 먹구 갑니다*^^*,너무 멋진 곳을 찾게 돼서 좋았습니다! 남자친구가 조용한 곳을 좋아해서 찾은 원테이...,음식들도 진짜 다 맛있는거 플러스 너무너무 친절하셔서 감동받았어용 자세하면서도 깔끔...,특별한날 좋은 추억을 만들로 가보세요 .,친절하시고 특별한 날 가기 좋아요,조용하고 차분한 분위기에 특별한날 방문하기 좋아요. \n처음 상담부터 너무 친절함 ...,36,58,['특별하다' '친절하다' '좋다' '용산' '분위기']
7661,18717349,청파서점,https://search.pstatic.net/common/?autoRotate=...,마포구,서점,서점,서울 용산구 청파로63길 3,신촌·이대역,5.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.600000,0.400000,0.000000,0.000000,시설이 깔끔해요,5.0,신간이 빨리 들어와요,4.0,책 종류가 다양해요,3.0,굳,ㄱ굿,맛,친절하고 좋았습니다~,ㅎ,NaN,NaN,NaN,NaN,NaN,0,18,no result
10701,1161694916,쎄임더스트,https://search.pstatic.net/common/?autoRotate=...,강남구,서점,독립서점,서울 성동구 성수일로 44-1 2층,압구정로데오거리,5.0,070-7954-1217,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.062500,0.750000,0.000000,0.187500,NaN,0.0,NaN,0.0,NaN,0.0,흔하지 않아서 좋고 친절해요,강동구에 이런 책방이라니 행운입니다!,이곳에서만 볼 수 있는 사진서적을 만날 수 있는 특별한 곳,지난번에 방문했다가 책이랑 티셔츠 사갔었는뎅 입은거 보고 친구가 예쁘다구해서 칭구네...,외국인아저씨가 한국말도 잘함 걱정말고 들어가도 될듯ㅋㅋㅋ 신기한 책이 많고 굿즈도 귀여움,흥미로운 포토북도 많고 매장도 깔끔하고 예뻐요!,한국에서 접하기 어려운 여러나라 사진작가들의 책들이 모여있는 곳! 볼게 너무 많아서...,NaN,NaN,NaN,9,14,['예쁘다']
10369,1327957454,소요서가,https://search.pstatic.net/common/?autoRotate=...,종로구,서점,서점,서울 중구 청계천로 160 청계상가 3층 바열 309-310호,종로·청계 관광특구,5.0,02-2272-1517,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.100000,0.600000,0.150000,0.150000,독특한 책이 많아요,15.0,시설이 깔끔해요,14.0,원하는 책을 찾기 쉬워요,10.0,대림상가에 서점이라니~ 깔끔한 내부가 훤히 보이는 작은 서점. 주로 철학책을 취급...,좋아요,"철학관련 서적을 전문으로 파는 서점입니다.철학입문서 추천 받았고요, 매우 친절하게 ...",흥미로운 책들이 많아요!,덕분에 원하는 책을 구할 수 있었어요 :)\n소요서가 정말 최고예요 !,신간 빨리 들여오시는 편입니다 취향맞으면 정말 이렇게 좋은 서점이없지요 매번 감사합니다,공간 아담하고 좋아요,철학위주의 책들이 많아 저에게는 어려웠지만 대중적인 책들을 골랐어요. 자체 책갈피를...,굿,각종 철학 서적들이 즐비해요! 사유를 즐기는 분들에게 특별한 장소가 되어 줄 듯 🥰...,34,32,['철학' '좋다' '서점']
3308,1244736538,옥상팩토리,https://search.pstatic.net/common/?autoRotate=...,강남구,전시관,전시관,"서울 송파구 법원로4길 5 지하1층 B111호, B112호, B113호",강남 MICE 관광특구,5.0,0507-1479-2334,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.150000,0.700000,0.100000,0.050000,NaN,0.0,NaN,0.0,NaN,0.0,원래는 입장료가 있지만 현재 진행중인 패치앤스티치 전시는 송파구의 지원을 받아서 무...,갤러리랑 같이운영해서그런지\n전체적으로 음료 퀄리티가 좋아요! 먹어본 것들 다 맛있...,<선택된 이야기> 전시에서는 글과 화화의 만남으로 나온 작업들을 감상할수 있었습니다...,특색있는 작업들을 하는 작가들을 모아 전시중이네요~! 전시 재밌어요~✨,지금은 <explorer reporting> 3인전이 진행중입니다. 각 작업에 담긴...,이번 전시는 김채윤 작가 개인전이었어요~! 공간기획도 재밌고 작가가 수집한 이미지들...,이번전시는 작가3인의 작업이 조화롭고 신비로운 느낌이 있고 음악이 함께해서 분위기가...,"김채윤 작가님 전시 보고 왔어요! 전시 너무 잘 봤고, 작가님께 직접 설명도 들을 ...",굿,이번전시에서 개성넘치는 작가들의 작업들을 볼수있어서 좋았습니다~! 비비드한 색감의 ...,91,51,['좋다' '전시' '재밌다' '작업' '작가']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2726,1725122120,슬슬 쇼룸,https://search.pstatic.net/common/?autoRotate=...,강남구,전시관,전시관,서울 강남구 삼성로122길 37,강남 MICE 관광특구,0.0,010-3321-9899,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000,0.000000,0.000000,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,no result
2727,33158565,꿈나무어린이공원,https://search.pstatic.net/common/?autoRotate=...,용산구,공원,공원,서울 용산구 이촌동,이촌한강공원,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.000000,0.000000,0.000000,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,no result
2728,19890790,국제스마트그리드및전기설비전,https://search.pstatic.